In [ ]:
!pip install langchain huggingface_hub tiktoken
!pip install chromadb
!pip install PyPDF2 InstructorEmbedding
!pip install --upgrade together

In [ ]:
import os

os.environ["TOGETHER_API_KEY"] = ""

In [ ]:
!pip show langchain

In [ ]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

In [ ]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.1
    """What sampling temperature to use."""

    max_tokens: int = 1024
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [ ]:
!pip install sentence-transformers==2.2.2

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [ ]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


In [ ]:
!pip install gdown

In [ ]:
import gdown

co_hamachar_zaky_file_id="1d2Y_rjcU8wqQ8sMsk7i1cizcfnUF0rdb"
url = f"https://drive.google.com/uc?id={co_hamachar_zaky_file_id}"
output_file = "co_hamachar_zaky.pdf"
gdown.download(url, output_file, quiet=False)

co_william_stallings_file_id="1UEHIoEMBg_1o8b5ckFvTdAZmrTyAVqAj"
url = f"https://drive.google.com/uc?id={co_william_stallings_file_id}"
output_file = "co_william_stallings.pdf"
gdown.download(url, output_file, quiet=False)

In [ ]:
from PyPDF2 import PdfReader

co_hamachar_zaky_reader=PdfReader('/content/co_hamachar_zaky.pdf')
co_william_stallings_reader=PdfReader('/content/co_william_stallings.pdf')

In [ ]:
!pip install wordninja

In [ ]:
import wordninja

start_and_end={"co_hamachar_zaky":[292,347,co_hamachar_zaky_reader],"co_william_stallings":[146,249,co_william_stallings_reader]}

mapping={}
all_chunks=[]

for key in start_and_end:
    start_page=start_and_end[key][0]
    end_page=start_and_end[key][1]

    for page_number in range(start_page - 1, end_page, 1):
        page = start_and_end[key][2].pages[page_number]
        temp_text=" ".join(wordninja.split(page.extract_text()))
        temp_chunks=RecursiveCharacterTextSplitter(
                chunk_size=1024, chunk_overlap=250, length_function=len
            ).split_text(temp_text)
        mapping[key+"_"+str(page_number+1)]=temp_chunks
        all_chunks.extend(temp_chunks)

In [ ]:
len(all_chunks)

In [ ]:
vectordb = Chroma.from_texts(texts=all_chunks,embedding=instructor_embeddings)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [ ]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature = 0.8,
    max_tokens = 2000
)

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))

In [ ]:
# full example

base_query="Explain me memory architecure"

ans=vectordb.similarity_search_with_relevance_scores(query=base_query, k=10,score_threshold=0.6)

for item in ans:
    for page in mapping:
        if str(item[0])[14:-1] in mapping[page]:
            print(item[1],page,'->',str(item[0])[14:-1],end='\n\n')

In [ ]:
llm_response = qa_chain(base_query)
process_llm_response(llm_response)